<a href="https://colab.research.google.com/github/Rai90/ggepy-tutorial/blob/main/gge_py_5_array_operation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee

# Trigger the authentication flow.
ee.Authenticate()



In [2]:
import geemap
ee.Initialize(project='ee-raischwalbert')

# **Bands in de axis**

Applying .toArray over a multiband ee.Image turns it into a single band image with the band values stored in a array format

![Image](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*O-f6NHpoW6vvv1EyUmYdzg.png)

# **Images in the axis**


Applying .toArray over a ee.ImageCollection turns it into a single band image where each pixel store a 2D array (image x bands)


![Image](https://miro.medium.com/v2/resize:fit:1100/format:webp/1*AvIw9Cjoc30b_s65akKvAQ.png)

# **Example: regression of year versus NDVI**


matrixSolve:
Solves for x in the matrix equation A * x = B, finding a least-squares solution if A is overdetermined for each matched pair of bands in image1 and image2. If either image1 or image2 has only 1 band, then it is used against all the bands in the other image.

In [4]:
# Define the start date and position to get images covering Montezuma Castle,
# Arizona, from 2000-2010.
start = '2000-01-01'
end = '2010-01-01'
lng = -111.83533
lat = 34.57499
region = ee.Geometry.Point(lng, lat)

# Filter to Landsat 7 images in the given time and place, filter to a regular
# time of year to avoid seasonal affects, and for each image create the bands
# we will regress on:
# 1. A 1, so the resulting array has a column of ones to capture the intercept.
# 2. Fractional year past 2000-01-01.
# 3. NDVI.

def data_prep(image):
  date = ee.Date(image.get('system:time_start'));
  yearOffset = date.difference(ee.Date(start), 'year');
  ndvi = image.normalizedDifference(['B4', 'B3']);
  return ee.Image(1).addBands(yearOffset).addBands(ndvi).toDouble()

images = ee.ImageCollection('LANDSAT/LE07/C02/T1') \
  .filterDate(start, end)\
  .filter(ee.Filter.dayOfYear(160, 240))\
  .filterBounds(region)\
  .map(lambda x: data_prep(x))

# display(images)


Map= geemap.Map()
Map.addLayer(ee.Image(images.first()))
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [5]:


# Convert to an array. Give the axes names for more readable code.
array = images.toArray()
imageAxis = 0
bandAxis = 1

Map.addLayer(array)
Map

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [6]:

x = array.arraySlice(bandAxis, 0, 2)
y = array.arraySlice(bandAxis, 2)
# display(x)
# display(y)


fit = x.matrixSolve(y)

# display(fit)

#Get the coefficient for the year, effectively the slope of the long-term NDVI trend
# as the resulting array has 2x1 dim the solpe is one the second row and first column
slope = fit.arrayGet([1, 0])

vis_params = {'bands': ['array'], 'min': -0.023, 'max': 0.025, 'opacity': 1.0, 'gamma': 1.0}

Map2 = geemap.Map()
Map2.setCenter(lng, lat, 12)
Map2.addLayer(slope,  vis_params, 'Slope')
Map2



Map(center=[34.57499, -111.83533], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…